In [1]:
dic = {}
dic['south-central'] = '13.65.92.139'
dic['west1'] = "13.93.236.162"
dic['central'] = "localhost"
def get_percentile(file_name, percentile):
    from collections import defaultdict
    d = defaultdict(list)
    with open(file_name) as f:
        for line in f:
            l = line.split()
            if len(l) == 3:
                key = l[0] + l[1]
                d[key].append(int(l[2]))
            elif len(l) == 4:
                key = l[0] + l[1] + l[2]
                d[key].append(int(l[3]))
            else:
                d[l[0]].append(int(l[1]))
    dic = {}
    import numpy as np
    for key, value in d.iteritems():
        a = np.array(value)
#         print np.amax(a)
        p = np.percentile(a, percentile)/1000
        dic[key] = p
    return dic

def data_breakdown_row(storage_name, percentile):
    thrift_latency = get_percentile("giza_trace.log", percentile)['Put_Thrift_Storage_' + dic[storage_name]]
    azure_latency = get_percentile(storage_name + "_storage_server.log", percentile)['Put_Azure_us-' + storage_name]
    difference = thrift_latency - azure_latency
    return str(thrift_latency) + "  |  " + str(azure_latency) + "  |  " + str(difference)
    
def metadata_breakdown_row(storage_name, percentile):
    thrift_latency = get_percentile("giza_trace.log", percentile)['Accept_Phase_thrfit_Table_us-' + storage_name]
    azure_latency = get_percentile(storage_name + "_table_server.log", percentile)['Accept_Phase_AzureTable_us-' + storage_name]
    difference = thrift_latency - azure_latency
    return str(thrift_latency) + "  |  " + str(azure_latency) + "  |  " + str(difference)


In [2]:
percentile = 95
size = '4mb'
f = 'performance_log/giza_latency_put_' + size + '.txt.log'
print 'Put              : ' + str(get_percentile(f, percentile)['Put'])
print 'Data Path     Put: ' + str(get_percentile(f, percentile)['SendErasureEncode'])
print 'Central       Put: ' + data_breakdown_row('central', percentile)
print 'SCentral      Put: ' + data_breakdown_row('south-central', percentile)
print 'West1         Put: ' + data_breakdown_row('west1', percentile)
print '----------------------------------------------------------'
get_version_latency = get_percentile(f, percentile)['GetLocalHighestCommittedVersionNumber']
fast_paxos_latency = get_percentile(f, percentile)['DoPhase2']
print 'Metadata Path Put: ' + str(get_version_latency + fast_paxos_latency)
print 'Get Highest Version: ' + str(get_version_latency)
print 'Do Fast Paxos     : ' + str(fast_paxos_latency)
print 'Central        Put: ' + metadata_breakdown_row('central', percentile)
print 'SCentral       Put: ' + metadata_breakdown_row('south-central', percentile)
print 'West1          Put: ' + metadata_breakdown_row('west1', percentile)


Put              : 298.58265
Data Path     Put: 295.57425
Central       Put: 101.53095  |  97.33925  |  4.1917
SCentral      Put: 250.06775  |  197.31035  |  52.7574
West1         Put: 230.56035  |  157.4101  |  73.15025
----------------------------------------------------------
Metadata Path Put: 211.82555
Get Highest Version: 17.015
Do Fast Paxos     : 194.81055
Central        Put: 77.8629  |  77.59505  |  0.26785
SCentral       Put: 161.7613  |  137.66535  |  24.09595
West1          Put: 186.80755  |  148.78985  |  38.0177


In [3]:
def data_breakdown_row(storage_name, percentile):
    thrift_latency = get_percentile("giza_trace.log", percentile)['GET_Thrift_Storage_' + dic[storage_name]]
    azure_latency = get_percentile(storage_name + "_storage_server.log", percentile)['Get_Azure_us-' + storage_name]
    difference = thrift_latency - azure_latency
    return str(thrift_latency) + "  |  " + str(azure_latency) + "  |  " + str(difference)
    
def metadata_breakdown_row(storage_name, percentile):
    thrift_latency = get_percentile("giza_trace.log", percentile)['Get_Highest_Version_Thrift_Table_us-' + storage_name]
    azure_latency = get_percentile(storage_name + "_table_server.log", percentile)['Get_Highest_Version_Number_AzureTable_us-' + storage_name]
    difference = thrift_latency - azure_latency
    return str(thrift_latency) + "  |  " + str(azure_latency) + "  |  " + str(difference)

percentile = 95
size = '4mb'
f = 'performance_log/giza_latency_get_' + size + '.txt.log'
print 'Get:               ' + str(get_percentile(f, percentile)['mainGet'])
print 'Data Path     Get: ' + str(get_percentile(f, percentile)['GetErasureEncode'])
print 'Central       Get: ' + data_breakdown_row('central', percentile)
print 'SCentral      Get: ' + data_breakdown_row('south-central', percentile)
print '----------------------------------------------------------'
get_version_latency = get_percentile(f, percentile)['GetLocal']
print 'Get Local Highest Version: ' + str(get_version_latency)
print 'Get Global Highest Version: ' + str(get_percentile(f, percentile)['GetGlobalHighestCommittedVersion'])
print 'Central        Get: ' + metadata_breakdown_row('central', percentile)
print 'SCentral       Get: ' + metadata_breakdown_row('south-central', percentile)

IOError: [Errno 2] No such file or directory: 'performance_log/giza_latency_get_4mb.txt.log'

In [ ]:
def get_std(file_name, percentile):
    from collections import defaultdict
    d = defaultdict(list)
    with open(file_name) as f:
        for line in f:
            l = line.split()
            if len(l) == 3:
                key = l[0] + l[1]
                d[key].append(int(l[2]))
            elif len(l) == 4:
                key = l[0] + l[1] + l[2]
                d[key].append(int(l[3]))
            else:
                d[l[0]].append(int(l[1]))
    dic = {}
    import numpy as np
    for key, value in d.iteritems():
        a = np.array(value)
#         print np.amax(a)
        p = np.std(a, percentile)/1000
        dic[key] = p
    return dic